# Assignment 4 – CSCN8030

# Adding imports

In [4]:
import pandas as pd

# Load the dataset

In [ ]:
# # Loading the Loblaws customer churn dataset.
df = pd.read_excel("./data/loblaws.xlsx")
df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Data,Variable,Discerption
1,NaN,E Comm,CustomerID,Unique customer ID
2,NaN,E Comm,Churn,Churn Flag
3,NaN,E Comm,Tenure,Tenure of customer in organization
4,NaN,E Comm,PreferredLoginDevice,Preferred login device of customer


# Exploratory Data Analysis

In [ ]:
# Summary statistics for numerical columns.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  0 non-null      float64
 1   Unnamed: 1  21 non-null     object 
 2   Unnamed: 2  21 non-null     object 
 3   Unnamed: 3  21 non-null     object 
dtypes: float64(1), object(3)
memory usage: 804.0+ bytes


In [ ]:
# Visualizing the churn distribution to understand imbalance.
df.describe()

,Unnamed: 0
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


# Identifing the numerical and categorical columns